# 🧠 Genie AI – Automated Marketing Content Workflow

### 📋 Overview

This notebook demonstrates a **fully automated workflow** for generating multi-channel marketing content for Genie AI’s upcoming **feature launch.**

It simulates a real-world internal marketing automation system that:
1. **Ingests mixed-format source material** (DOCX, PDF meeting notes, customer feedback, and engineering tickets).
2. **Summarizes and consolidates** all key information into a unified product launch brief using OpenAI’s `responses` API.
3. **Generates tailored marketing assets** (LinkedIn post, Newsletter email, and Blog post) with consistent voice and channel-appropriate tone.
4. **Exports the outputs** into Markdown for easy publishing or integration into downstream automations (e.g., CMS, email platform, social scheduling tools).

The workflow is designed to be:
- **Repeatable** → Drop new product docs in the `docs_in/` folder and re-run to regenerate launch assets.
- **Sustainable** → Modular architecture separates ingestion, summarization, and generation for future re-use.
- **Automation-ready** → Easily extendable into Zapier, Make, or n8n pipelines.

---

### Future Extensions

This marketing pipeline can be extended into a full automation system by integrating with tools like **Slack, Email, LinkedIn, and CMS platforms**. Drafts could be automatically posted to Slack for team review and approval, then—once approved—sent as newsletters via services like SendGrid or Mailchimp, published as LinkedIn posts through the LinkedIn API, or uploaded directly to a blog CMS such as Ghost or WordPress. These extensions would transform the current script from a content generator into an **end-to-end AI-driven marketing automation pipeline**, streamlining creation, review, and publishing across multiple channels.

In [2]:
from dotenv import load_dotenv

# 1. Imports + config
from ingest import ingest_documents
from summarise import build_prompt
from generate import build_payload
import os
import dotenv
load_dotenv()
from openai import OpenAI

client = OpenAI()

SUMMARY_MODEL = "gpt-4o-mini"
ASSET_MODEL = "gpt-4o-mini"
SUMMARY_TEMPERATURE = 0.3
ASSET_TEMPERATURE = 0.5
SUMMARY_MAX_TOKENS = 2000
ASSET_MAX_TOKENS = 1400

## Ingest the documents

In [3]:
docs, lookup = ingest_documents()

print(f"Loaded {len(docs)} docs.")
for d in docs:
    print(f"- {d['name']}: {len(d['text'])} chars")


Loaded 5 docs.
- TASK  Customer feedback snippets: 357 chars
- TASK  Linear Ticket   Engineering: 925 chars
- TASK  Marketing & Product Meeting Notes: 1318 chars
- TASK  Meeting Transcript (Product + Engineering + Marketing): 1575 chars
- TASK  Product Roadmap Summary: 511 chars


### Build prompt

The prompt is built by collating all sources into the prompt.

### Use prompt to get summary of all docs_in

In [4]:
summary_payload = build_prompt()
summary_response = client.responses.create(
      model=SUMMARY_MODEL,
      instructions=summary_payload["system"],
      input=[{"role": "user", "content": summary_payload["user"]}],
      temperature=SUMMARY_TEMPERATURE,
      max_output_tokens=SUMMARY_MAX_TOKENS,
  )
summary_output = summary_response.output_text.strip()
summary_output[:600]  # preview

'### Launch Brief for Document Compare Feature\n\n#### 1. Executive Summary\nThe Document Compare feature is designed to streamline the review process for legal documents by providing users with a clear, efficient way to identify changes between versions. This native tool enhances productivity for founders and executives, allowing them to focus on critical content rather than getting lost in tedious revisions.\n\n#### 2. Customer Insights\n- **Pain Points:**\n  - Users struggle with manually scrolling through multiple documents, leading to frustration (Source 1).\n  - The complexity of redlines is daun'

## Create a post for each type of asset: LinkedIn, Mail, and Blog

In [5]:
def generate_asset(content_type: str, launch_brief: str) -> str:
  payload = build_payload(content_type, launch_brief)
  resp = client.responses.create(
      model=ASSET_MODEL,
      instructions=payload["system"],
      input=[{"role": "user", "content": payload["user"]}],
      temperature=ASSET_TEMPERATURE,
      max_output_tokens=ASSET_MAX_TOKENS,
  )
  return resp.output_text.strip()


In [6]:
assets = {}
for t in ["linkedin", "newsletter", "blog"]:
    print(f"Generating {t}…")
    assets[t] = generate_asset(t, summary_output)

print("\nLINKEDIN:\n", assets["linkedin"][:200], "...\n")
print("NEWSLETTER:\n", assets["newsletter"][:200], "...\n")
print("BLOG:\n", assets["blog"][:200], "...\n")

Generating linkedin…
Generating newsletter…
Generating blog…

LINKEDIN:
 **🔍 Simplify Your Document Review Process!**

Tired of sifting through endless legal documents? Our new Document Compare feature is here to save you time and sanity. Instantly identify changes between ...

NEWSLETTER:
 ### Subject Line Options:
1. "Streamline Your Document Reviews Today!"
2. "Revolutionize Legal Document Comparisons!"

### Preview Text:
Discover Genie AI's new feature for effortless document reviews ...

BLOG:
 # Streamline Your Legal Review Process with Genie AI's Document Compare Feature

**Meta Description:** Discover how Genie AI's Document Compare feature can save you time and money while ensuring secur ...



In [20]:
from pathlib import Path

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

(OUT_DIR / "launch_brief.md").write_text(summary_output + "\n", encoding="utf-8")
for kind, text in assets.items():
  path = OUT_DIR / f"{kind}.md"
  path.write_text(text + "\n", encoding="utf-8")
  print(f"Saved {kind} to {path}")

✅ Saved Markdown to /Users/matth/local_software_projects/feature-release-marketing/outputs/GenieAI_Marketing_Outputs_2025-11-11_18-08.md


PosixPath('outputs/GenieAI_Marketing_Outputs_2025-11-11_18-08.md')

In [7]:
from pathlib import Path
from export import markdown_file_to_docx

out_dir = Path("outputs")
for md_file in out_dir.glob("*.md"):
  docx_path = out_dir / f"{md_file.stem}.docx"
  markdown_file_to_docx(md_file, docx_path)
  print(f"Saved {docx_path}")


Saved outputs/newsletter.docx
Saved outputs/linkedin.docx
Saved outputs/blog.docx
Saved outputs/GenieAI_Marketing_Outputs_2025-11-11_18-08.docx
Saved outputs/launch_brief.docx
Saved outputs/GenieAI_Marketing_Outputs.docx
